In [1]:
import os
import mne
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
from mne.time_frequency import psd_array_multitaper

In [2]:
# Path to the dataset root
data_dir = './ds003523'

# Initialize lists for features and labels
all_features = []
all_labels = []

# Define frequency bands
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}


# Load participants.tsv to get the Group labels
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')



In [3]:
participants_file = os.path.join(data_dir, 'participants.tsv')
participants = pd.read_csv(participants_file, sep='\t')
participants

,participant_id,Original_ID,URSI,sex,age,Group
0,sub-001,3044,M87102477,1,23,0
1,sub-002,3009,M87104171,0,30,1
2,sub-003,3042,M87104582,0,19,0
3,sub-004,3074,M87106278,0,19,0
4,sub-005,3076,M87107075,0,22,0
...,...,...,...,...,...,...
86,sub-087,3084,M87192041,0,34,0
87,sub-088,3063,M87192146,1,30,1
88,sub-089,3082,M87196019,0,24,0
89,sub-090,3007,M87196158,1,26,1


In [4]:
participants.Group.value_counts()

Group
0    57
1    34
Name: count, dtype: int64

### Helper Functions

In [5]:
# Function to compute additional features
def compute_additional_features(epoch_data):
    """
    Computes additional features like:
    - Mean, variance, skewness, and kurtosis for time-domain data
    - Line length (signal complexity)
    """
    from scipy.stats import skew, kurtosis

    # Mean and variance across time points
    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)

    # Skewness and kurtosis across time points
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)

    # Line length (sum of absolute differences between consecutive samples)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

In [6]:
# X = np.vstack(all_features)  # Features for all subjects and sessions
# y = np.hstack(all_labels)  

In [7]:
# Function to compute additional features (mean, variance, skewness, kurtosis, line length)
def compute_additional_features(epoch_data):
    if len(epoch_data.shape) != 2:
        raise ValueError(f"Expected 2D array for epoch_data, got {epoch_data.shape}")

    mean_values = epoch_data.mean(axis=1)
    var_values = epoch_data.var(axis=1)
    skew_values = skew(epoch_data, axis=1)
    kurtosis_values = kurtosis(epoch_data, axis=1)
    line_length = np.sum(np.abs(np.diff(epoch_data, axis=1)), axis=1)

    return mean_values, var_values, skew_values, kurtosis_values, line_length

# Initialize variables
all_features = []
all_labels = []
all_subjects = []  # To store subject IDs for each epoch
band_ranges = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 100)
}
expected_feature_count = 10  # Total number of features (5 frequency + 5 time-domain)

# Loop through each subject
    # Loop through each subject
for subject in range(0, 92):  # Adjust range as needed
    subject_id = f"sub-{subject:03d}"  # Format: sub-001, sub-002, etc.
    subject_id_lower = subject_id.lower()
    
    # Condition to skip a specific subject
    if subject_id_lower == "sub-056":
        continue  # Skip this iteration
    # Condition to skip a specific subject
    if subject_id_lower == "sub-077":
        continue  # Skip this iteration
    # Condition to skip a specific subject
    if subject_id_lower == "sub-079":
        continue  # Skip this iteration

    # Your code for processing other subjects
    print(f"Processing {subject_id_lower}")

    # Check if subject exists in participants.tsv
    if subject_id_lower not in participants['participant_id'].values:
        print(f"Subject {subject_id} not found in participants.tsv. Skipping.")
        continue

    # Locate the subject directory
    subject_dir = os.path.join(data_dir, subject_id)
    if not os.path.exists(subject_dir):
        print(f"Subject directory {subject_id} not found. Skipping.")
        continue

    # Find all available sessions for the subject
    #sessions = [s for s in os.listdir(subject_dir) if s.startswith('ses-')]
    for session in ["ses-01"]:
        session_path = os.path.join(subject_dir, session, 'eeg')




    for session in ["ses-01"]:
        session_path = os.path.join(subject_dir, session, 'eeg')

        # Retrieve session data using DataLad
        os.system(f'datalad get {session_path}')

        # Check if the .set file exists for this session
        eeg_file = os.path.join(session_path, f"{subject_id}_{session}_task-VisualWorkingMemory_eeg.set")
        if not os.path.exists(eeg_file):
            print(f"Data for {subject_id}, {session} not found. Skipping.")
            continue

        # Load the EEG data
        raw = mne.io.read_raw_eeglab(eeg_file, preload=True)

        # Preprocess the data
        raw.filter(0.5, 100., fir_design='firwin')

        # Extract events
        events, event_id = mne.events_from_annotations(raw)

        # Define epochs
        tmin, tmax = -0.2, 0.5
        epochs = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)

        # Compute PSD for the entire frequency range
        psd = epochs.compute_psd(method='multitaper', fmin=0.5, fmax=100)
        psd_data = psd.get_data()  # Shape: (n_epochs, n_channels, n_freqs)
        freqs = psd.freqs

        # Compute features for each frequency band
        band_powers = {}
        for band, (fmin, fmax) in band_ranges.items():
            band_indices = (freqs >= fmin) & (freqs < fmax)
            band_power = psd_data[:, :, band_indices].mean(axis=2)  # Mean PSD within the band
            band_powers[band] = band_power.mean(axis=1)  # Aggregate across channels

        # Extract time-domain data from epochs
        epoch_data = epochs.get_data().mean(axis=1)  # Aggregate across channels (shape: n_epochs, n_times)

        # Compute additional features
        mean_values, var_values, skew_values, kurtosis_values, line_length = compute_additional_features(epoch_data)

        # Combine all features into a single array
        try:
            session_features = np.column_stack([
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['gamma'], mean_values,
                var_values, skew_values, kurtosis_values, line_length
            ])
            print(f"Session features shape for {subject_id}, {session}: {session_features.shape}")
        except ValueError as e:
            print(f"Feature dimension mismatch for {subject_id}, {session}: {e}")
            continue

        # Append subject ID for each epoch
        subject_ids = np.repeat(subject_id, session_features.shape[0])

        all_features.append(session_features)
        all_subjects.append(subject_ids)

        # Get the group label for the subject
        group_label = participants.loc[participants['participant_id'] == subject_id_lower, 'Group'].values
        if len(group_label) == 0 or pd.isna(group_label[0]):
            print(f"No valid group label found for {subject_id}. Skipping.")
            continue

        # Repeat the label for each epoch in the session
        all_labels.append(np.repeat(group_label[0], session_features.shape[0]))

Processing sub-000
Subject sub-000 not found in participants.tsv. Skipping.
Processing sub-001


[ERROR] NoDatasetFound(No dataset found at '/Users/ishabharti/EEG Project' for the purpose "get content of <<['./ds003523/s++15 chars++eg']>>".  Specify a dataset to work with by providing its path via the `dataset` option, or change the current working directory to be in a dataset.) (NoDatasetFound) 
usage: datalad get [-h] [-s LABEL] [-d PATH] [-r] [-R LEVELS] [-n]
                   [-D DESCRIPTION] [--reckless [auto|ephemeral|shared-...]]
                   [-J NJOBS] [--version]
                   [PATH ...]


Reading /Users/ishabharti/EEG Project/ds003523/sub-001/ses-01/eeg/sub-001_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 697174  =      0.000 ...  1394.348 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (6.602 s)



/var/folders/dl/zd_s8q0x20zbxpy0_dzhksbh0000gn/T/ipykernel_68894/1038661286.py:78: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Used Annotations descriptions: ['S  1', 'S  2', 'S  3', 'S 50', 'S 51', 'S 52', 'S100', 'S101', 'S200', 'S201', 'boundary']
Not setting metadata
511 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 511 events and 351 original time points ...
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Session features shape for sub-001, ses-01: (510, 10)
Processing sub-002
Reading /Users/ishabharti/EEG Project/ds003523/sub-002/ses-01/eeg/sub-002_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 766849  =      0.000 ...  1533.698 secs...


...

Used Annotations descriptions: ['S  1', 'S  2', 'S  3', 'S 50', 'S 51', 'S 52', 'S100', 'S101', 'S200', 'S201', 'boundary']
Not setting metadata
511 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 511 events and 351 original time points ...
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Session features shape for sub-090, ses-01: (510, 10)
Processing sub-091
Reading /Users/ishabharti/EEG Project/ds003523/sub-091/ses-01/eeg/sub-091_ses-01_task-VisualWorkingMemory_eeg.fdt
Reading 0 ... 720499  =      0.000 ...  1440.998 secs...


[ERROR] NoDatasetFound(No dataset found at '/Users/ishabharti/EEG Project' for the purpose "get content of <<['./ds003523/s++15 chars++eg']>>".  Specify a dataset to work with by providing its path via the `dataset` option, or change the current working directory to be in a dataset.) (NoDatasetFound) 
usage: datalad get [-h] [-s LABEL] [-d PATH] [-r] [-R LEVELS] [-n]
                   [-D DESCRIPTION] [--reckless [auto|ephemeral|shared-...]]
                   [-J NJOBS] [--version]
                   [PATH ...]


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (6.602 s)



/var/folders/dl/zd_s8q0x20zbxpy0_dzhksbh0000gn/T/ipykernel_68894/1038661286.py:78: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Used Annotations descriptions: ['S  1', 'S  2', 'S  3', 'S 50', 'S 51', 'S 52', 'S100', 'S101', 'S200', 'S201', 'boundary']
Not setting metadata
507 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 507 events and 351 original time points ...
1 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Session features shape for sub-091, ses-01: (506, 10)


In [8]:
# Combine all features, labels, and subject IDs
if not all_features or not all_labels:
    print("No features or labels collected. Exiting.")
    raise ValueError("No features or labels collected.")

X = np.vstack(all_features)
y = np.hstack(all_labels)
subject_ids = np.hstack(all_subjects)

# Save the data with subject IDs
feature_columns = [
    'delta', 'theta', 'alpha', 'beta', 'gamma',
    'mean', 'variance', 'skewness', 'kurtosis', 'line_length'
]
training_data = pd.DataFrame(X, columns=feature_columns)
training_data['label'] = y
training_data['subject_id'] = subject_ids
training_data.to_csv('eeg_training_data_with_subjects.csv', index=False)

In [9]:
training_data.head(100000)

,delta,theta,alpha,beta,gamma,mean,variance,skewness,kurtosis,line_length,label,subject_id
0,0.000123,0.000067,1.039174e-05,4.101177e-07,1.740069e-07,-0.000003,2.089062e-08,-1.763948,2.470575,0.004479,0,sub-001
1,0.000208,0.000085,9.628139e-07,2.513081e-07,1.123781e-07,0.000125,4.599112e-08,0.497322,-1.170764,0.003333,0,sub-001
2,0.000917,0.000410,3.276360e-05,4.333310e-06,3.227772e-07,0.000167,1.393151e-07,0.577952,-0.144130,0.005464,0,sub-001
3,0.000069,0.000030,1.999117e-06,3.742573e-07,1.669777e-07,-0.000084,1.022358e-08,0.009396,-1.516320,0.003864,0,sub-001
4,0.000231,0.000103,7.266782e-06,3.089532e-06,2.499983e-07,0.000274,4.820566e-08,-0.132097,-0.538502,0.004247,0,sub-001
...,...,...,...,...,...,...,...,...,...,...,...,...
44362,0.000013,0.000007,9.744368e-07,5.313773e-07,1.449815e-06,0.000044,4.604022e-09,-0.090585,-0.633751,0.012132,0,sub-091
44363,0.000120,0.000066,1.038919e-05,6.017075e-07,1.444217e-06,-0.000086,1.725815e-08,-0.060559,0.376205,0.012171,0,sub-091
44364,0.000022,0.000011,1.920983e-06,2.185408e-06,1.516083e-06,0.000031,7.818443e-09,0.996139,1.549020,0.012272,0,sub-091
44365,0.000006,0.000004,7.138447e-07,5.277213e-07,1.427296e-06,0.000015,4.114224e-09,-0.093129,-0.464614,0.011401,0,sub-091
